# covariates , tagsを作成する

In [1]:
# すでにcomplete_tags,complete_death_tagsがあるのでUNIQUE_IDとsample_idを合わせておわり

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import datetime
from datetime import timedelta
import numpy as np

from tqdm import tqdm
import time

file_path="D:/mimic_iii/csv/"
outfile_path="D:/mimic_iii/clean_data/"

In [2]:
df=pd.read_csv(outfile_path+'GRU_ODE_Dataset.csv')

In [3]:
tags=pd.read_csv(outfile_path+"complete_death_tags.csv")
covs=pd.read_csv(outfile_path+"complete_covariates.csv")

In [4]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [5]:
# ロード
d=pickle_load( outfile_path+'sample_unique_id_dict.pickle')

In [6]:
# sample_id->unique_idをunique_id->sample_idに
# 
def get_swap_dict(d):
    return {v: k for k, v in d.items()}

d_swap = get_swap_dict(d)
print(d_swap)

778, 3968: 19779, 7696: 19780, 6211: 19781, 14100: 19782, 11454: 19783, 4845: 19784, 11639: 19785, 5662: 19786, 15353: 19787, 7319: 19788, 17171: 19789, 19477: 19790, 9703: 19791, 2344: 19792, 16154: 19793, 5656: 19794, 3049: 19795, 14564: 19796, 7515: 19797, 7875: 19798, 3440: 19799, 5325: 19800, 19445: 19801, 9744: 19802, 20084: 19803, 21005: 19804, 12409: 19805, 17191: 19806, 9016: 19807, 8953: 19808, 16989: 19809, 14092: 19810, 21141: 19811, 5703: 19812, 1486: 19813, 4411: 19814, 513: 19815, 396: 19816, 4593: 19817, 7140: 19818, 15317: 19819, 15861: 19820, 20900: 19821, 16742: 19822, 7779: 19823, 19471: 19824, 17088: 19825, 8327: 19826, 47: 19827, 18074: 19828, 7907: 19829, 20786: 19830, 3950: 19831, 8094: 19832, 9412: 19833, 1188: 19834, 7001: 19835, 14012: 19836, 19259: 19837, 7281: 19838, 5973: 19839, 16284: 19840, 16366: 19841, 4783: 19842, 15499: 19843, 19378: 19844, 21122: 19845, 18867: 19846, 7592: 19847, 18644: 19848, 10685: 19849, 10333: 19850, 20323: 19851, 13578: 19852, 

# tags process

In [7]:
tags

,Unnamed: 0,HADM_ID,Value,UNIQUE_ID
0,5524,123550,0,0
1,4559,119261,0,1
2,10356,144283,0,2
3,15479,166402,0,3
4,4112,117437,0,4
...,...,...,...,...
21256,20218,186216,0,21256
21257,3374,114388,0,21257
21258,2254,109751,0,21258
21259,4727,120075,0,21259


In [8]:
if "Unnamed: 0" in tags.columns:
    tags=tags.drop("Unnamed: 0",axis=1)
if "HADM_ID" in tags.columns:
    tags=tags.drop("HADM_ID",axis=1)
tags

,Value,UNIQUE_ID
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
21256,0,21256
21257,0,21257
21258,0,21258
21259,0,21259


In [9]:
tags=tags.reindex(columns=['UNIQUE_ID', 'Value'])
tags

,UNIQUE_ID,Value
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
21256,21256,0
21257,21257,0
21258,21258,0
21259,21259,0


In [10]:
tags["UNIQUE_ID"]=tags["UNIQUE_ID"].map(d_swap)
tags=tags.rename(columns={'UNIQUE_ID' : 'ID' , "Value":"label"})

In [11]:
tags

,ID,label
0,7513,0
1,10119,0
2,259,0
3,20874,0
4,3093,0
...,...,...
21256,2345,0
21257,4213,0
21258,20657,0
21259,10688,0


In [12]:
tags.to_csv(outfile_path+"GRU_ODE_death_tags.csv",index=False)

In [13]:
tags.columns

Index(['ID', 'label'], dtype='object')

## Covs process

In [14]:
covs

,UNIQUE_ID,004,008,009,011,012,013,021,027,031,...,997,998,999,V45,V50,V55,V56,V58,V59,V71
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21256,21256,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21257,21257,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21258,21258,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21259,21259,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
covs["UNIQUE_ID"]=covs["UNIQUE_ID"].map(d_swap)
covs=covs.rename(columns={'UNIQUE_ID' : 'ID'})

In [16]:
covs

,ID,004,008,009,011,012,013,021,027,031,...,997,998,999,V45,V50,V55,V56,V58,V59,V71
0,7513,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10119,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,259,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20874,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3093,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21256,2345,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21257,4213,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21258,20657,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21259,10688,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
covs.to_csv(outfile_path+"GRU_ODE_covariates.csv",index=False)

In [18]:
import torch
print(torch.cuda.is_available())

True


In [19]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(dev)


cuda:0


In [20]:
torch.cuda.get_device_name(0)

'GeForce RTX 3070'

In [21]:
 torch.backends.cudnn.version()

8004

In [22]:
m = torch.nn.Sigmoid()
input = torch.randn(3,1)
output = m(input)

In [23]:
input

tensor([[-2.0267],
        [ 0.5111],
        [ 2.2622]])

In [24]:
output

tensor([[0.1164],
        [0.6251],
        [0.9057]])

In [25]:
input = torch.randn(3,1)
output = torch.sigmoid(input)

In [26]:
output

tensor([[0.6876],
        [0.5922],
        [0.8287]])

In [27]:
output.shape

torch.Size([3, 1])